- https://www.worldclim.org/data/worldclim21.html
- Citation:

  Fick, S.E. and R.J. Hijmans, 2017. WorldClim 2: new 1km spatial resolution
  climate surfaces for global land areas. International Journal of Climatology
  37 (12): 4302-4315.


In [ ]:
import xarray as xr

import fsspec

In [ ]:
month_vars = ["prec", "srad", "tavg", "tmax", "tmin", "vapr", "wind"]

months = xr.Variable("month", list(range(1, 13)))

ds = xr.Dataset()

for var in month_vars:
    ds[var] = xr.concat(
        [
            xr.open_rasterio(
                f"gs://carbonplan-scratch/worldclim-raw/wc2.1_30s_{var}/wc2.1_30s_{var}_{m:02d}.tif",
                chunks={"x": 8192, "y": 8192},
            ).squeeze(drop=True)
            for m in months.values
        ],
        dim=months,
        compat="override",
        coords="minimal",
    )

In [ ]:
ds

In [ ]:
ds["elev"] = xr.open_rasterio(
    f"gs://carbonplan-scratch/worldclim-raw/wc2.1_30s_elev.tif",
    chunks={"x": 8192, "y": 8192},
).squeeze(drop=True)

In [ ]:
bio_names = {
    "BIO1": "Annual Mean Temperature",
    "BIO2": "Mean Diurnal Range (Mean of monthly (max temp - min temp))",
    "BIO3": "Isothermality (BIO2/BIO7) (×100)",
    "BIO4": "Temperature Seasonality (standard deviation ×100)",
    "BIO5": "Max Temperature of Warmest Month",
    "BIO6": "Min Temperature of Coldest Month",
    "BIO7": "Temperature Annual Range (BIO5-BIO6)",
    "BIO8": "Mean Temperature of Wettest Quarter",
    "BIO9": "Mean Temperature of Driest Quarter",
    "BIO10": "Mean Temperature of Warmest Quarter",
    "BIO11": "Mean Temperature of Coldest Quarter",
    "BIO12": "Annual Precipitation",
    "BIO13": "Precipitation of Wettest Month",
    "BIO14": "Precipitation of Driest Month",
    "BIO15": "Precipitation Seasonality (Coefficient of Variation)",
    "BIO16": "Precipitation of Wettest Quarter",
    "BIO17": "Precipitation of Driest Quarter",
    "BIO18": "Precipitation of Warmest Quarter",
    "BIO19": "Precipitation of Coldest Quarter",
}


for bionum in range(1, 20):
    ds[f"BIO{bionum:02d}"] = xr.open_rasterio(
        f"gs://carbonplan-scratch/worldclim-raw/wc2.1_30s_bio/wc2.1_30s_bio_{bionum}.tif",
        chunks={"x": 8192, "y": 8192},
    ).squeeze(drop=True)
    ds[f"BIO{bionum:02d}"].attrs["description"] = bio_names[f"BIO{bionum:d}"]

In [ ]:
mapper = fsspec.get_mapper("gs://carbonplan-data/raw/worldclim/30s/raster.zarr")

ds.to_zarr(mapper, consolidated=True)